<a href="https://colab.research.google.com/github/mohanvamsi1999/curly-octo-lamp/blob/master/ASRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pulp
import itertools
from math import floor


# Define the set of ASIs and their associated sampling rates
ASIs = ['ASI1', 'ASI2', 'ASI3','ASI4']
sampling_rates = [8e3, 12e3, 16e3, 24e3, 32e3, 48e3, 96e3, 192e3]
sampling_rates = [(44.1/48)*ii for ii in sampling_rates] + sampling_rates

# Define the cost constraint
cost_constraint = 1
num_channels = 16
DSP_clk = 100e6
num_iter = 0
max_buffer_size = 0

# Initialize lists to store valid transformations and the current transformation set
max_buffer_transformations = {}
max_values = {}

# Function to calculate the cost of a transformation
def calculate_cost1(input_rate, output_rate):
    if output_rate > input_rate:
      if output_rate < 8*input_rate:
        zeta = 1/DSP_clk
      else:
        zeta = 1
    else:
      if output_rate > input_rate/8:
        zeta = (input_rate / output_rate)/DSP_clk
      else:
        zeta = 1
    return output_rate*140*zeta

# Function to calculate the cost of a transformation
def calculate_cost2(input_rate, output_rate):
    if output_rate > input_rate:
        zeta = 1/DSP_clk
    else:
        zeta = (input_rate / output_rate)/DSP_clk
    return  140*zeta


# Function to calculate the cost of all the 12 possible transformations, given the 4 rates f1, f2, f3, f4 using calculate cost function
# Also, let us use itertools.permutations to get all the possible permutations of the 4 rates

def calculate_all_costs(f1, f2, f3, f4):
    # list of costs
    costs = {}
    for fx, fy in itertools.permutations([f1, f2, f3, f4], 2):
        # append to the list of costs
        costs[(fx,fy)] = [calculate_cost1(fx, fy), calculate_cost2(fx, fy)]
    return costs

all_combinations = itertools.combinations_with_replacement(sampling_rates, 4)

for f1,f2,f3,f4 in all_combinations:
    num_iter = num_iter + 1
    costs = calculate_all_costs(f1, f2, f3, f4)
    # Coefficients of the objective function
    p = {k: v[1] for k, v in costs.items()}

    # Coefficients of the inequality constraint
    w = {k: v[0] for k, v in costs.items()}

    # Define the problem
    prob = pulp.LpProblem("The Problem", pulp.LpMaximize)

    # Decision variables
    x = pulp.LpVariable.dicts("x", costs.keys(), 0, None, cat='Integer')

    # Objective function
    prob += pulp.lpSum([p[i]*x[i] for i in costs.keys()])

    # Constraints
    prob += pulp.lpSum([w[i]*x[i] for i in costs.keys()]) <= cost_constraint
    prob += pulp.lpSum([x[i] for i in costs.keys()]) <= num_channels

    # Solve the problem
    status = prob.solve()

    if pulp.LpStatus[status] == 'Optimal':
        objective_value = pulp.value(prob.objective)
        #print("Objective Value: ", objective_value)
        max_values[(f1, f2, f3, f4)] = objective_value
        max_buffer_transformations[(f1, f2, f3, f4)] = {i: pulp.value(x[i]) for i in costs.keys()}


for asi_freq in max_values:
  buffer_size = 0
  for freq in asi_freq:
    buffer_size += floor(freq*max_values[asi_freq]) + 1
  if buffer_size > max_buffer_size:
    max_buffer_size = buffer_size


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [3]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 27.1 MB/s eta 0:00:00


In [5]:
max_buffer_size


59

In [7]:
# Find the key with the maximum value in the dictionary
max_key = max(max_values, key=lambda k: max_values[k])

# Find the maximum value itself
max_value = max_values[max_key]

In [8]:
max_key

(7350.000000000001, 7350.000000000001, 7350.000000000001, 44100.0)

In [9]:
max_value

0.0001344

In [13]:
import pulp
import itertools
from math import floor


# Define the set of ASIs and their associated sampling rates
ASIs = ['ASI1', 'ASI2', 'ASI3','ASI4']
sampling_rates = [8e3, 12e3, 16e3, 24e3, 32e3, 48e3, 96e3, 192e3]
sampling_rates = [(44.1/48)*ii for ii in sampling_rates] + sampling_rates

# Define the cost constraint
cost_constraint = 1
num_channels = 16
DSP_clk = 100e6
num_iter = 0
max_buffer_size = 0
max_service_time = 135e-6

# Initialize lists to store valid transformations and the current transformation set
max_buffer_transformations = {}
max_values = {}

# Function to calculate the cost of a transformation
def calculate_cost1(input_rate, output_rate):
    if output_rate > input_rate:
      if output_rate < 8*input_rate:
        zeta = 1/DSP_clk
      else:
        zeta = 1
    else:
      if output_rate > input_rate/8:
        zeta = (input_rate / output_rate)/DSP_clk
      else:
        zeta = 1
    return output_rate*140*zeta

# Function to calculate the cost of a transformation
def calculate_cost2(input_rate, output_rate):
    if output_rate > input_rate:
        zeta = 1/DSP_clk
    else:
        zeta = (input_rate / output_rate)/DSP_clk
    return  140*zeta

# Function to calculate the cost of a transformation
def calculate_cost3(input_rate, output_rate):
    num_taps = 67
    if output_rate > input_rate:
        filter_buf = num_taps + 1
    else:
        filter_buf = floor(num_taps*(input_rate/output_rate)) + 2
    return floor(input_rate*max_service_time)+floor(output_rate*max_service_time)+4+filter_buf

# Function to calculate the cost of all the 12 possible transformations, given the 4 rates f1, f2, f3, f4 using calculate cost function
# Also, let us use itertools.permutations to get all the possible permutations of the 4 rates

def calculate_all_costs(f1, f2, f3, f4):
    # list of costs
    costs = {}
    for fx, fy in itertools.permutations([f1, f2, f3, f4], 2):
        # append to the list of costs
        costs[(fx,fy)] = [calculate_cost1(fx, fy), calculate_cost2(fx, fy), calculate_cost3(fx,fy)]
    return costs

all_combinations = itertools.combinations_with_replacement(sampling_rates, 4)

for f1,f2,f3,f4 in all_combinations:
    num_iter = num_iter + 1
    costs = calculate_all_costs(f1, f2, f3, f4)

    # Coefficients of the objective function
    q = {k: v[2] for k, v in costs.items()}

    # Coefficients of the objective function
    p = {k: v[1] for k, v in costs.items()}

    # Coefficients of the inequality constraint
    w = {k: v[0] for k, v in costs.items()}

    # Define the problem
    prob = pulp.LpProblem("The Problem", pulp.LpMaximize)

    # Decision variables
    x = pulp.LpVariable.dicts("x", costs.keys(), 0, None, cat='Integer')

    # Objective function uses p to find service time and q for memory size
    prob += pulp.lpSum([q[i]*x[i] for i in costs.keys()])

    # Constraints
    prob += pulp.lpSum([w[i]*x[i] for i in costs.keys()]) <= cost_constraint
    prob += pulp.lpSum([x[i] for i in costs.keys()]) <= num_channels

    # Solve the problem
    status = prob.solve()

    if pulp.LpStatus[status] == 'Optimal':
        objective_value = pulp.value(prob.objective)
        #print("Objective Value: ", objective_value)
        max_values[(f1, f2, f3, f4)] = objective_value
        max_buffer_transformations[(f1, f2, f3, f4)] = {i: pulp.value(x[i]) for i in costs.keys()}

# Find the key with the maximum value in the dictionary
max_key = max(max_values, key=lambda k: max_values[k])

# This is either the interrupt queue length or buffer memory needed.
max_value = max_values[max_key]

for asi_freq in max_values:
  buffer_size = 0
  for freq in asi_freq:
    buffer_size += floor(freq*max_values[asi_freq]) + 1
  if buffer_size > max_buffer_size:
    max_buffer_size = buffer_size


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [14]:
max_value

6679.0

In [15]:
max_buffer_transformations[max_key]

{(7350.000000000001, 29400.000000000004): 0.0,
 (7350.000000000001, 12000.0): 0.0,
 (7350.000000000001, 48000.0): 0.0,
 (29400.000000000004, 7350.000000000001): 1.0,
 (29400.000000000004, 12000.0): 0.0,
 (29400.000000000004, 48000.0): 0.0,
 (12000.0, 7350.000000000001): 1.0,
 (12000.0, 29400.000000000004): 0.0,
 (12000.0, 48000.0): 0.0,
 (48000.0, 7350.000000000001): 14.0,
 (48000.0, 29400.000000000004): 0.0,
 (48000.0, 12000.0): 0.0}